In [1]:
import sys
sys.path.append("./utils/")
import random
from data_path import *
from glob import glob
from os.path import join, basename, dirname, samefile
from tqdm import tqdm
from skimage import data, color, exposure 
from skimage.io import imread
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import accuracy_score,f1_score
from scipy.io import loadmat
import re
%load_ext autoreload
%autoreload 1
%aimport data_path

In [5]:
mat = loadmat('../Features/padoraPHoT.mat')
main = loadmat('../pandora18kMain_v2.mat')

In [6]:
X_hot = mat['data'][:18000]
Y_hot = mat['imgLabel'][:18000].flatten()

In [9]:
np.save(join(FEATURES['RF_HoT'], "X_hot_NN"), X_hot)
np.save(join(FEATURES['RF_HoT'], "Y_hot_NN"), Y_hot)

In [7]:
main_file_names = ['..\\Pandora_18k\\CROP\\' + f[0][0][0][0][0] for f in main['imageListFile'][:18000]]

In [73]:
X = list()
Y = list()
files = get_all_files(cropped=True)
for file in tqdm(files):
    image = imread(file)
    image = color.rgb2hsv(image)
    fd = exposure.histogram(image[:,:,0])[0]
    X.append(fd)
    Y.append(get_genre_id(file))
X = np.asarray(X)
Y = np.asarray(Y)
np.save(join(FEATURES["RF"],'X_hist'),X)
np.save(join(FEATURES["RF"],'Y'),Y)

100%|█████████████████████████████████████████████████████████████████████████| 334363/334363 [47:50<00:00, 116.50it/s]


In [10]:
files_cropped = get_all_files(cropped=True)
# files_origin = get_all_files(cropped=False)

In [13]:
X = np.load(join(FEATURES["RF"],'X_hist.npy'))
Y = np.load(join(FEATURES["RF"],'Y.npy'))

In [11]:
files_cropped_dict = dict()
for i,f in tqdm(enumerate(files_cropped)):
    directory = dirname(f)
    name_origin = re.sub("[0-9]+_[0-9]+_","",basename(f))
    try:
        files_cropped_dict[(directory, name_origin)].append(i)
    except KeyError:
        files_cropped_dict[(directory, name_origin)] = [i]

334363it [00:04, 78983.64it/s]


In [14]:
uncut_files = {(dirname(f),basename(f)) for f in main_file_names}
print(len(uncut_files))
to_remove = set()
for el in uncut_files:
    if el not in files_cropped_dict:
        to_remove.add(el)
uncut_files = uncut_files.difference(to_remove)
len(to_remove), len(uncut_files)        

18000


(47, 17953)

In [15]:
RF_train_files = set(random.sample(uncut_files,9000))
RF_test_files = uncut_files.difference(RF_train_files)

In [16]:
RF_train_indices = list()
for f in RF_train_files:
    RF_train_indices.extend(files_cropped_dict[f])
RF_train_indices = np.asarray(RF_train_indices)

In [17]:
X_train_RF = X[RF_train_indices]
Y_train_RF = Y[RF_train_indices]

In [18]:
RF_test_indices = list()
RF_test_files = sorted(RF_test_files)
for f in RF_test_files:
    RF_test_indices.extend(files_cropped_dict[f])
RF_test_indices = np.asarray(RF_test_indices)

In [19]:
X_test_RF = X[RF_test_indices]
Y_test_RF = Y[RF_test_indices]

In [20]:
np.save(join(FEATURES["RF_HoT"],'X_train_RF'),X_train_RF)
np.save(join(FEATURES["RF_HoT"],'Y_train_RF'),Y_train_RF)
np.save(join(FEATURES["RF_HoT"],'X_test_RF'),X_test_RF)
np.save(join(FEATURES["RF_HoT"],'Y_test_RF'),Y_test_RF)

In [17]:
X_train_RF = np.load(join(FEATURES["RF_HoT"],'X_train_RF.npy'))
Y_train_RF = np.load(join(FEATURES["RF_HoT"],'Y_train_RF.npy'))
X_test_RF = np.load(join(FEATURES["RF_HoT"],'X_test_RF.npy'))
Y_test_RF = np.load(join(FEATURES["RF_HoT"],'Y_test_RF.npy'))

In [21]:
classifier = RandomForestClassifier(n_estimators=100 )
classifier.fit(X_train_RF,Y_train_RF)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [22]:
import pickle
pickle.dump(classifier, open(join(FEATURES["RF_HoT"],'RF_classifier'), 'wb'))
pickle.dump(RF_test_files, open(join(FEATURES["RF_HoT"],'RF_test_files'), 'wb'))

In [3]:
import pickle
classifier = pickle.load( open(join(FEATURES["RF_HoT"],'RF_classifier'), 'rb'))
RF_test_files = pickle.load(open(join(FEATURES["RF_HoT"],'RF_test_files'), 'rb'))

In [5]:
X_test_RF = np.load(join(FEATURES["RF_HoT"],'X_test_RF.npy'))
Y_test_RF = np.load(join(FEATURES["RF_HoT"],'Y_test_RF.npy'))

In [8]:
RF_test_files = np.load(join(FEATURES["RF_HoT"],'RF_test_files.npy'))

In [14]:
X_RF_NN = list()
X_hot_NN = list()
Y_RF_NN = list()
Y_hot_NN = list()
for f in tqdm(RF_test_files):
    hot_index = main_file_names.index(join(f[0], f[1]))
    X_hot_NN.append(X_hot[hot_index])
    Y_hot_NN.append(Y_hot[hot_index])
    indices = files_cropped_dict[f]
    ys = classifier.predict(X[indices])
    ans = np.zeros((150, 18)).astype(int)
    i = 0
    for y in ys:
        ans[i][y-1] = 1
        i += 1
        if i >= 150:
            break
    X_RF_NN.append(ans.flatten())
    Y_RF_NN.append(Y[indices][0])
X_RF_NN = np.asarray(X_RF_NN)
Y_RF_NN = np.asarray(Y_RF_NN)
X_hot_NN = np.asarray(X_hot_NN)
Y_hot_NN = np.asarray(Y_hot_NN)

100%|█████████████████████████████████████████████████████████████████████████████| 8953/8953 [01:11<00:00, 124.88it/s]


In [15]:
X_RF_hot_NN = np.concatenate([X_RF_NN, X_hot_NN], axis=1)
Y_RF_hot_NN = Y_RF_NN

In [16]:
np.save(join(FEATURES['RF_HoT'], "X_RF_hot_NN_2"), X_RF_hot_NN)
np.save(join(FEATURES['RF_HoT'], "Y_RF_hot_NN_2"), Y_RF_hot_NN)